Acknowledge that the dataset herein is miniature and can be toyed with on Jupyter nb using a local machine, perhaps with single GPU. More realistic datasets would require API interfacing with a cluster and data engineering pipelines to display distributions of billions of params over time, or to run robust imputation methods for missing data as standalone endeavors before one even considers modeling.


In [155]:
import pandas as pd
import numpy as np

In [156]:
data = pd.read_csv("../data/train_sparked.csv")
X_test_nulls_raw = pd.read_csv("../data/test_sparked.csv")

In [157]:
data

,SellerCity,SellerIsPriv,SellerListSrc,SellerName,SellerRating,SellerRevCnt,SellerState,SellerZip,VehCertified,VehColorExt,...,VehListdays,VehMake,VehMileage,VehModel,VehPriceLabel,VehSellerNotes,VehYear,Vehicle_Trim,Dealer_Listing_Price,NumOwners
0,warren,False,inventorycommandcenter,primemotorz,5.0,32,MI,48091,False,white,...,8.600069,Jeep,39319,grandcherokee,fairprice,None.,2015,High Altitude,30990.0,1.0
1,fargo,False,cadillaccertifiedprogram,gatewaychevroletcadillac,4.8,1456,ND,58103,True,black,...,2.920127,Cadillac,30352,xt5,gooddeal,Come take a look at our great pre-owned invent...,2017,NaN,34860.0,1.0
2,waukesha,False,jeepcertifiedprogram,wildechryslerjeepdodgeramampsubaru,4.8,1405,WI,53186,True,brilliantblackcrystalpearlcoat,...,28.107014,Jeep,38957,grandcherokee,gooddeal,Backed by a rigorous 125-point inspection by f...,2015,Laredo,23249.0,1.0
3,wentzville,False,inventorycommandcenter,centurydodgechryslerjeepram,4.4,21,MO,63385,False,diamondblackcrystalpearlcoat,...,59.816875,Jeep,20404,grandcherokee,gooddeal,Drop by to see us and you will quickly see how...,2018,Limited,31977.0,1.0
4,fayetteville,False,homenetautomotive,superiorbuickgmcoffayetteville,3.7,74,AR,72703,False,radiantsilvermetallic,...,98.665301,Cadillac,19788,xt5,gooddeal,"Luxury, Exterior Parking Camera Rear, Front Du...",2018,Luxury,33495.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6289,dearborn,True,sellityourself,abe,0.0,0,MI,48126,False,black,...,29.781968,Jeep,49000,grandcherokee,NaN,****ALL BLACK EDITION****You are viewing a bea...,2015,High Altitude,18699.0,NaN
6290,indianapolis,False,digitalmotorworksdmi,carmaxindianapolis,3.3,16,IN,46280,False,gray,...,4.840069,Jeep,20039,grandcherokee,fairprice,CarMax makes car buying easy and hassle-free. ...,2015,Limited,31998.0,1.0
6291,dublin,False,digitalmotorworksdmi,cadillacofdublin,4.1,20,OH,43017,True,black,...,184.921991,Cadillac,16278,xt5,gooddeal,Clean CARFAX. Certified. Black 2018 Cadillac X...,2018,Luxury,35674.0,0.0
6292,sandusky,False,digitalmotorworksdmi,fosterchevroletcadillac,4.9,278,OH,44870,False,black,...,73.868426,Cadillac,38146,xt5,greatdeal,Black 2017 Cadillac XT5 Luxury FWD 8-Speed Aut...,2017,Luxury,31995.0,2.0


In [158]:
data.dtypes

SellerCity               object
SellerIsPriv               bool
SellerListSrc            object
SellerName               object
SellerRating            float64
SellerRevCnt              int64
SellerState              object
SellerZip                 int64
VehCertified               bool
VehColorExt              object
VehColorInt              object
VehDriveTrain            object
VehEngine                object
VehFeats                 object
VehFuel                  object
VehHistory               object
VehListdays             float64
VehMake                  object
VehMileage                int64
VehModel                 object
VehPriceLabel            object
VehSellerNotes           object
VehYear                   int64
Vehicle_Trim             object
Dealer_Listing_Price    float64
NumOwners               float64
dtype: object

In [159]:
# len(data["Word2Vec_VehSellerNotes"].unique()), len(data["Word2Vec_SellerName"].unique())

In [160]:
data.dtypes

SellerCity               object
SellerIsPriv               bool
SellerListSrc            object
SellerName               object
SellerRating            float64
SellerRevCnt              int64
SellerState              object
SellerZip                 int64
VehCertified               bool
VehColorExt              object
VehColorInt              object
VehDriveTrain            object
VehEngine                object
VehFeats                 object
VehFuel                  object
VehHistory               object
VehListdays             float64
VehMake                  object
VehMileage                int64
VehModel                 object
VehPriceLabel            object
VehSellerNotes           object
VehYear                   int64
Vehicle_Trim             object
Dealer_Listing_Price    float64
NumOwners               float64
dtype: object

In [161]:
X_test_nulls_raw

,ListingID,SellerCity,SellerIsPriv,SellerListSrc,SellerName,SellerRating,SellerRevCnt,SellerState,SellerZip,VehCertified,...,VehFuel,VehHistory,VehListdays,VehMake,VehMileage,VehModel,VehPriceLabel,VehSellerNotes,VehYear,NumOwners
0,8622015,seneca,False,homenetautomotive,lakekeoweechryslerdodgejeepram,2.5,59,SC,29678,False,...,Gasoline,"['non-personal use reported', 'buyback protect...",143.991262,Cadillac,13625.0,xt5,gooddeal,Thank you for visiting another one of Lake Keo...,2018,1.0
1,8625693,bedford,False,inventorycommandcenter,northcoastautomall,4.7,2116,OH,44146,False,...,Gasoline,"['accident(s) reported', 'non-personal use rep...",138.770486,Jeep,42553.0,grandcherokee,gooddeal,This 2017 Jeep Grand Cherokee 4dr Limited 4x4 ...,2017,1.0
2,8625750,webster,False,jeepcertifiedprogram,marinachryslerdodgejeepmitsubishiram,3.9,46,NY,14580,True,...,E85 Flex Fuel,['buyback protection eligible'],31.951088,Jeep,48951.0,grandcherokee,gooddeal,Certified. Brilliant Black Crystal Pearlcoat 2...,2015,1.0
3,8626885,louisville,False,digitalmotorworksdmi,oxmoorfordlincoln,4.5,1075,KY,40222,False,...,Gasoline,['buyback protection eligible'],5.950127,Jeep,44179.0,grandcherokee,gooddeal,2015 Jeep Grand Cherokee ***THIS VEHICLE IS AT...,2015,1.0
4,8627430,palmyra,False,digitalmotorworksdmi,fckerbeckampsons,4.6,162,NJ,8065,False,...,Gasoline,"['non-personal use reported', 'buyback protect...",24.672986,Cadillac,22269.0,xt5,gooddeal,"AWD, CarFax One Owner! Navigation, Back-up Cam...",2018,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9992442,forestpark,False,homenetautomotive,curriechevy,4.8,1081,IL,60130,False,...,Gasoline,['buyback protection eligible'],18.091597,Jeep,24744.0,grandcherokee,gooddeal,granite crystal metallic clearcoat 2017 Jeep G...,2017,1.0
996,9993562,tampa,False,inventorycommandcenter,tampamitsubishi,4.0,240,FL,33614,False,...,Gasoline,"['non-personal use reported', 'buyback protect...",167.799676,Cadillac,5699.0,xt5,gooddeal,Tampa Mitsubishi is proud to offer this attrac...,2017,1.0
997,9994646,tampa,False,homenetautomotive,fermanacura,5.0,134,FL,33612,False,...,Gasoline,"['accident(s) reported', 'non-personal use rep...",46.215625,Cadillac,17985.0,xt5,gooddeal,*BRAND NEW* *2018* *CADILLAC* *XT5** LOADED WI...,2018,1.0
998,9997199,hamburg,False,inventorycommandcenter,townechryslerdodgejeepraminc,3.8,7,NY,14075,False,...,Diesel,['buyback protection eligible'],14.907535,Jeep,27.0,grandcherokee,fairprice,"Thousand?s of Vehicles, Positively Different E...",2018,1.0


In [162]:
y_train_nulls = data[["Dealer_Listing_Price", "Vehicle_Trim"]]
y_train_null_id = y_train_nulls[data.isnull().any(axis=1)].index


X_train_nulls_raw = data.drop(
    ["Dealer_Listing_Price", "Vehicle_Trim"], axis=1
)  # nulls to indicate nulls present and raw to indicate ft are not numerically encoded yet
y_test_id_nulls_raw = X_test_nulls_raw["ListingID"]  # save for later
X_test_nulls_raw = X_test_nulls_raw.drop(["ListingID"], axis=1)

We note some features that only have two options and are binarized. Recall that OrdinalEncoder is suitable for categorical variables with a meaningful order. OneHotEncoder is suitable for categorical variables without a natural specific order and can be used to binarize. Seller rating is tricky to deal with as it's a numeric categorical feature as ratings are still an ordinal measure (what does 3.5 rating - 2.0 vs 1.0 + 5.0 rating even mean??) Before we tend to these pipelines simplify the codification of ext/int colors as there is a lot of repetition.


In [163]:
# Words to match and replace with the corresponding color name
colors_to_replace = [
    "blue",
    "red",
    "black",
    "silver",
    "white",
    "cashmere",
    "steel",
    "granite",
    "ivory",
    "amethyst",
    "green",
    "gray",
    "brown",
    "bronze",
    "auburn",
    "sangria",
    "mocha",
    "rhino",
]

# Constructing a regular expression pattern to match any portion containing the specified colors
pattern = "|".join(colors_to_replace)

# Replace any portion of the string containing 'blue', 'red', or 'black' etc with only the corresponding color name
# The r'\1' in the value parameter is a backreference to the matched portion, so it replaces the entire string with only the color name.
X_train_nulls_raw["VehColorExt"] = X_train_nulls_raw["VehColorExt"].replace(
    to_replace=f".*({pattern}).*", value=r"\1", regex=True
)
X_test_nulls_raw["VehColorExt"] = X_test_nulls_raw["VehColorExt"].replace(
    to_replace=f".*({pattern}).*", value=r"\1", regex=True
)

print(
    X_train_nulls_raw["VehColorExt"].unique(),
    len(X_train_nulls_raw["VehColorExt"].unique()),
)
print(
    X_test_nulls_raw["VehColorExt"].unique(),
    len(X_test_nulls_raw["VehColorExt"].unique()),
)

['white' 'black' 'silver' 'blue' 'red' 'granite' 'gray' 'brown' 'steel'
 'charcoal' 'velvet' 'green' 'bronze' 'rhino' nan 'ivory' 'amethyst'
 'sangria' 'maroon' 'cashmere' 'certifiedhemilthrpanoroofnav' 'purple'
 'certifiedlthrroofnavcamera' 'burgundy' 'auburn' 'gold' 'grey' 'pearl'
 'diamond' 'other' 'midnightsky' 'beige' 'tan' 'notspecified'
 'certifiedlthrpanoroofnavcamera' 'platinum' 'pewter' 'mocha'
 'shadowmetallic' 'gy' 'certifiedlthrpanoroofhotcoldseats' 'pink' 'billet'
 'certifiedroofcamerahtdseats' 'beigh' 'midnightskymetallic' 'unspecified'
 'brightsil'] 48
['black' 'blue' 'granite' 'silver' 'white' nan 'red' 'beige' 'amethyst'
 'gray' 'ivory' 'billet' 'rhino' 'deepcherry' 'sangria' 'bronze' 'green'
 'velvet' 'steel' 'cashmere' 'brown' 'auburn' 'grey' 'midnightsky'
 'burgundy' 'shadowmetallic' 'tan' 'notspecified' 'diamond' 'undetermined'
 'mocha'] 31


In [164]:
X_train_nulls_raw["VehColorInt"] = X_train_nulls_raw["VehColorInt"].replace(
    to_replace=f".*({pattern}).*", value=r"\1", regex=True
)
X_test_nulls_raw["VehColorInt"] = X_test_nulls_raw["VehColorInt"].replace(
    to_replace=f".*({pattern}).*", value=r"\1", regex=True
)

print(
    X_train_nulls_raw["VehColorExt"].unique(),
    len(X_train_nulls_raw["VehColorInt"].unique()),
)
print(
    X_test_nulls_raw["VehColorExt"].unique(),
    len(X_test_nulls_raw["VehColorInt"].unique()),
)

# Careful not to overreach and drop colors you do not see in the test set as it's meant to be beyond our purview. This was simply to cut down on complexity

['white' 'black' 'silver' 'blue' 'red' 'granite' 'gray' 'brown' 'steel'
 'charcoal' 'velvet' 'green' 'bronze' 'rhino' nan 'ivory' 'amethyst'
 'sangria' 'maroon' 'cashmere' 'certifiedhemilthrpanoroofnav' 'purple'
 'certifiedlthrroofnavcamera' 'burgundy' 'auburn' 'gold' 'grey' 'pearl'
 'diamond' 'other' 'midnightsky' 'beige' 'tan' 'notspecified'
 'certifiedlthrpanoroofnavcamera' 'platinum' 'pewter' 'mocha'
 'shadowmetallic' 'gy' 'certifiedlthrpanoroofhotcoldseats' 'pink' 'billet'
 'certifiedroofcamerahtdseats' 'beigh' 'midnightskymetallic' 'unspecified'
 'brightsil'] 38
['black' 'blue' 'granite' 'silver' 'white' nan 'red' 'beige' 'amethyst'
 'gray' 'ivory' 'billet' 'rhino' 'deepcherry' 'sangria' 'bronze' 'green'
 'velvet' 'steel' 'cashmere' 'brown' 'auburn' 'grey' 'midnightsky'
 'burgundy' 'shadowmetallic' 'tan' 'notspecified' 'diamond' 'undetermined'
 'mocha'] 22


In [165]:
X_train_nulls_raw.columns, X_train_nulls_raw.shape, X_test_nulls_raw.shape

(Index(['SellerCity', 'SellerIsPriv', 'SellerListSrc', 'SellerName',
        'SellerRating', 'SellerRevCnt', 'SellerState', 'SellerZip',
        'VehCertified', 'VehColorExt', 'VehColorInt', 'VehDriveTrain',
        'VehEngine', 'VehFeats', 'VehFuel', 'VehHistory', 'VehListdays',
        'VehMake', 'VehMileage', 'VehModel', 'VehPriceLabel', 'VehSellerNotes',
        'VehYear', 'NumOwners'],
       dtype='object'),
 (6294, 24),
 (1000, 24))

Now we focus on the text columns and use Word2Vec to study them. I left notes in spark.py if you want more details on rationale (doing it on this end as converting pyspark Vector type is hassle)


In [166]:
X_train_nulls_raw["VehSellerNotes"]

0                                                   None.
1       Come take a look at our great pre-owned invent...
2       Backed by a rigorous 125-point inspection by f...
3       Drop by to see us and you will quickly see how...
4       Luxury, Exterior Parking Camera Rear, Front Du...
                              ...                        
6289    ****ALL BLACK EDITION****You are viewing a bea...
6290    CarMax makes car buying easy and hassle-free. ...
6291    Clean CARFAX. Certified. Black 2018 Cadillac X...
6292    Black 2017 Cadillac XT5 Luxury FWD 8-Speed Aut...
6293    With Vroom, you can buy your next car from the...
Name: VehSellerNotes, Length: 6294, dtype: object

In [167]:
enumerate(X_train_nulls_raw["VehSellerNotes"])

In [168]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
import nltk
import re

# https://radimrehurek.com/gensim/models/doc2vec.html
# https://www.geeksforgeeks.org/nlp-gensim-tutorial-complete-guide-for-beginners/

nltk.download("stopwords")
stop_words = set(stopwords.words("english"))


def tokenize(df, col):
    df[col] = df[col].fillna("none")
    df[col] = df[col].replace("", "none")

    # remove punctuation and special characters alongside stop words
    df[col] = df[col].apply(
        lambda text: [
            re.sub(r"[^a-zA-Z0-9]", "", token.lower())
            for token in word_tokenize(text)
            if token.lower() not in stop_words
        ]
    )

    # removing any resulting blank space entries like in  ['arsenic', '', 'cat'] --> ['arsenic', 'cat']
    df[col] = df[col].apply(lambda text: " ".join(text).split())

    # now each entry (document) of the col has words in our custom tokenized format:
    """
    DF now looks something like this (train shown here):
    0                                               [none]
    1    [come, take, look, great, preowned, inventory,...
    2    [backed, rigorous, 125point, inspection, facto...
    3    [drop, see, us, quickly, see, century, x27, sp...
    4    [luxury, exterior, parking, camera, rear, fron...
    .
    .
    .
    """

def tagDoc(df, col, tokens_only=False):
    tokenize(df, col)
    for i, line in enumerate(df[col]):
        print(line)
        if tokens_only:
            yield line
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(line, [i])


def trainedWordModel(train, col, vector_size=100, min_count=3, epochs=40):
    model = gensim.models.doc2vec.Doc2Vec(
        vector_size=vector_size, min_count=min_count, epochs=epochs
    )
    train_corpus = tagDoc(train, col, tokens_only=False)

    # Essentially, the vocabulary is a list (accessible via model.wv.index_to_key)
    # of all of the unique words extracted from the training corpus.
    model.build_vocab(train_corpus)

    model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

    # Now, we can use the trained model to infer a vector for any piece of text by passing a list of words to the model.infer_vector function

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gbert\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [169]:
train_corpus = list(tagDoc(X_train_nulls_raw, "VehSellerNotes", tokens_only=False))

In [ ]:
print(train_corpus[:2])

TypeError: 'generator' object is not subscriptable

In [ ]:
trainedWordModel(X_train_nulls_raw, "VehSellerNotes")

TypeError: expected string or bytes-like object, got 'list'

Since we just are encoding here there is no danger in leakage (not true if adding ML model predictions as another step)


In [ ]:
X_nulls = pd.concat([X_train_nulls_raw, X_test_nulls_raw])
X_nulls.shape

In [ ]:
X_nulls.columns, len(X_nulls.columns)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Define column subsets
categorical_cols = [
    "SellerIsPriv",
    "SellerListSrc",
    "SellerState",
    "SellerZip",
    "VehCertified",
    "VehColorExt",
    "VehColorInt",
    "VehDriveTrain",
    "VehEngine",
    "VehFeats",
    "VehFuel",
    "VehHistory",
    "VehMake",
    "VehModel",
]
ordinal_cols = ["SellerRating", "VehPriceLabel"]
numeric_cols = ["SellerRevCnt", "VehListdays", "VehMileage", "VehYear", "NumOwners"]

# Create transformers
categorical_transformer = Pipeline(
    steps=[
        # ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "onehot",
            OneHotEncoder(handle_unknown="infrequent_if_exist", sparse_output=False),
        ),
    ]
)

ordinal_transformer = Pipeline(
    steps=[
        # ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "ordinal",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan),
        ),
    ]
)

numeric_transformer = Pipeline(
    steps=[
        # ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=True)),
    ]  # https://stackoverflow.com/questions/52008548/python-running-into-x-test-y-test-fit-errors
)

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_cols),
        ("ord", ordinal_transformer, ordinal_cols),
        ("num", numeric_transformer, numeric_cols),
    ],
    remainder="passthrough",  # NOTE CRITICAL -- leave the other features not explicitly called as is but make sure all cols accounted for
)

# Create a final pipeline with the preprocessor and any subsequent steps (e.g., a model)
final_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        # Add additional steps as needed, e.g., a machine learning model -- holding off as we want to do imputation study
    ]
)
X = final_pipeline.fit_transform(X_nulls)

In [ ]:
np.any([arrayOfArrayElement is np.nan for arrayOfArrayElement in X])

In [ ]:
X.shape

In [ ]:
X_train_nulls = X[: len(X_train_nulls_raw)]  # drop raw as now encoded
X_test_nulls = X[len(X_train_nulls_raw) :]
X_train_nulls.shape, X_test_nulls.shape

In [ ]:
original_array = np.zeros(3, dtype=object)

# Example NumPy array to assign to the first index
array_to_assign = np.array([1, 2, 3], dtype=object)

# Example string list to assign to the second index
list_to_assign = 3

# Assign the NumPy array to the first index
original_array[0] = array_to_assign

# Assign the string list to the second index
original_array[1] = list_to_assign

In [ ]:
original_array

In [ ]:
np.any([arrayOfArrayElement is np.nan for arrayOfArrayElement in original_array])

In [ ]:
pd.DataFrame(X_train_nulls)

In [ ]:
X_train_nulls[3206]

Now that we have encoded the data let's take a look at the distributions (without the compression from encoding)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

distCol = [
    "VehListdays",
    "VehMileage",
    "NumOwners",
    "SellerRating",
    "SellerRevCnt",
]
# Set up subplots -- NOTE: Dealer_Listing_Price is part of what we want to predict!
fig, axes = plt.subplots(nrows=len(distCol), ncols=1, figsize=(8, 4 * len(distCol)))

## Plot distribution and statistics for each column
for i, column in enumerate(distCol):
    # Plot distribution
    sns.histplot(X_train_nulls_raw[column], kde=True, ax=axes[i])

    # Calculate statistics
    mean_value = X_train_nulls_raw[column].mean()
    median_value = X_train_nulls_raw[column].median()
    std_dev_value = X_train_nulls_raw[column].std()

    # Plot major statistics below the distribution plot
    axes[i].axvline(
        mean_value,
        color="red",
        linestyle="dashed",
        linewidth=2,
        label=f"Mean ({mean_value:.2f})",
    )
    axes[i].axvline(
        median_value,
        color="green",
        linestyle="dashed",
        linewidth=2,
        label=f"Median ({median_value:.2f})",
    )
    axes[i].axvline(
        mean_value + std_dev_value,
        color="orange",
        linestyle="dashed",
        linewidth=2,
        label=f"Std Dev ({std_dev_value:.2f})",
    )
    axes[i].axvline(
        mean_value - std_dev_value, color="orange", linestyle="dashed", linewidth=2
    )
    axes[i].legend()

    # Set x-axis limits to the minimum and maximum values of the distribution
    axes[i].set_xlim(X_train_nulls_raw[column].min(), X_train_nulls_raw[column].max())

    # Set plot labels and title
    axes[i].set_xlabel(column)
    axes[i].set_ylabel("Frequency")
    axes[i].set_title(f"Distribution of {column}")

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Set up subplots
fig, ax = plt.subplots(figsize=(8, 4))

# Plot distribution
sns.histplot(y_train_nulls["Dealer_Listing_Price"], kde=True, ax=ax)

# Calculate statistics
mean_value = y_train_nulls["Dealer_Listing_Price"].mean()
median_value = y_train_nulls["Dealer_Listing_Price"].median()
std_dev_value = y_train_nulls["Dealer_Listing_Price"].std()

# Plot major statistics below the distribution plot
ax.axvline(
    mean_value,
    color="red",
    linestyle="dashed",
    linewidth=2,
    label=f"Mean ({mean_value:.2f})",
)
ax.axvline(
    median_value,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label=f"Median ({median_value:.2f})",
)
ax.axvline(
    mean_value + std_dev_value,
    color="orange",
    linestyle="dashed",
    linewidth=2,
    label=f"Std Dev ({std_dev_value:.2f})",
)
ax.axvline(mean_value - std_dev_value, color="orange", linestyle="dashed", linewidth=2)
ax.legend()

# Set x-axis limits to the minimum and maximum values of the distribution
ax.set_xlim(
    y_train_nulls["Dealer_Listing_Price"].min(),
    y_train_nulls["Dealer_Listing_Price"].max(),
)

# Set plot labels and title
ax.set_xlabel("Dealer_Listing_Price")
ax.set_ylabel("Frequency")
ax.set_title("Distribution of Dealer_Listing_Price")

# Show the plot
plt.show()

None of the data distributions look Gaussian but standard scaling will be used as it will not change the shape of the distribution.

https://stats.stackexchange.com/questions/453211/using-standardscaler-function-of-scikit-learn-library
https://stats.stackexchange.com/questions/290958/logistic-regression-and-scaling-of-features


At this point it is time to impute missing data before we drop certain rows due to outliers, say, outside 3 std dev from in-column averages. We shall use a random forest model (less sensitive to outliers present) as a baseline model for imputation though other NN-backed methods can be explored and explore various imputers.

https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html#sphx-glr-auto-examples-impute-plot-missing-values-py


First want to get a score on a dataset which had no missing values


Let's start by getting the data into numerical input schemes for missing data (nulls). To do this and avoid data leakage, we need the statistics calculated for each column are calculated on the training dataset only, then applied to the train and test sets for each fold in the dataset under consideration. For the sake of time we will use a Random Forest regressor as the baseline imputation model. Other approaches can develop a NN-backed approach instead or use other ensemble methods.

https://scikit-learn.org/stable/auto_examples/impute/plot_missing_values.html#sphx-glr-auto-examples-impute-plot-missing-values-py
